### This notebook allows the user to create a directory structure and locally upload the metadata to the MILE2 repository.

In [13]:
import pandas as pd
import os
from os import walk
import shutil
from ipywidgets import *
import ipywidgets as widgets
import requests
from contextlib import closing
import csv
import io
from IPython.display import display

### Now let's select some metadata.

Create a list of subdirectories in the collection directory of MILE2 to select metadata for evaluation

In [14]:
def OrganizationChoices(organization):
    global OrganizationChoice
    global Organization
    Organization=organization
    print("Curating Organization", Organization)
    
def CollectionName(collection):
    global CollectionChoice
    global Collection
    Collection=collection
    print("Collection is named", Collection)
    
def DialectName(dialect):
    global DialectChoice
    global Dialect
    Dialect=dialect
    print("Collection is written in the", Dialect,"dialect")   
    
def metadataLocation(directory):
    global LocationChoice
    global MetadataLocation
    MetadataLocation=directory
    print("The local directory", MetadataLocation,"contains the metadata for evaluation")    
    
w=interactive(OrganizationChoices, organization='')    

w2=interactive(CollectionName, collection='')

w3=interactive(DialectName, dialect='')

w4=interactive(metadataLocation, directory='')    

display(w)
display(w2)
display(w3)
display(w4)

Copy the metadata to the new directory structure.

In [38]:
%cd ../zip
MetadataDestination=os.path.join(Organization,Collection,Dialect,'xml')
os.makedirs(MetadataDestination, exist_ok=True)
os.makedirs(os.path.join('../data',Organization), exist_ok=True)
src_files = os.listdir(MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

/Users/scgordon/MetadataEvaluation/zip


'/Users/scgordon/MetadataEvaluation/upload/metadata.zip'

In [39]:
%cd ../upload 
# Send metadata package, read the response into a dataframe
url = 'http://metadig.nceas.ucsb.edu/metadata/evaluator'
files = {'zipxml': open('metadata.zip', 'rb')}
r = requests.post(url, files=files, headers={"Accept-Encoding": "gzip"})
r.raise_for_status()
EvaluatedMetadataDF = pd.read_csv(io.StringIO(r.text), quotechar='"')

#build filepaths and file names
Filedirectory=os.path.join('../data/',Organization)

Filename='/'+Collection+'_'+Dialect+'_Evaluated.csv.gz'
SimplfiedFilename='/'+Collection+'_'+Dialect+'_EvaluatedSimplified.csv.gz'
FilePath=Filedirectory+Filename
SimplifiedFilePath=Filedirectory+SimplfiedFilename
EvaluatedMetadataDF.insert(3, 'Collection', Collection+'_'+Dialect)

EvaluatedMetadataDF.to_csv(FilePath, mode = 'w', compression='gzip', index=False)

#Change directories, delete upload directory and zip. Delete copied metadata.
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(Organization)
%cd ../data

#Create a simplified XPath output
EvaluatedSimplifiedMetadataDF = EvaluatedMetadataDF.copy()
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/gco:CharacterString', '')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[a-z]+:+?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[A-Z]+_[A-Za-z]+/?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('//', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.rstrip('//')
EvaluatedSimplifiedMetadataDF.to_csv(SimplifiedFilePath, mode = 'w', compression='gzip', index=False)

/Users/scgordon/MetadataEvaluation/upload
/Users/scgordon/MetadataEvaluation
/Users/scgordon/MetadataEvaluation/zip
/Users/scgordon/MetadataEvaluation/data


In [41]:
FiledirectoryRAD=os.path.join('../data/',Organization)
FilenameRAD='/'+Collection+'_'+Dialect+'_RAD.csv'
FilePathRAD=FiledirectoryRAD+FilenameRAD
group_name = EvaluatedSimplifiedMetadataDF.groupby(['Collection','Record', 'Concept'], as_index=False)
occuranceMatrix=group_name.size().unstack().reset_index()
occuranceMatrix=occuranceMatrix.fillna(0)
occuranceMatrix.columns.names = ['']
pd.options.display.float_format = '{:,.0f}'.format
occuranceMatrix.to_csv(FilePathRAD, mode = 'w', index=False)
occuranceMatrix

,Collection,Record,Abstract,Acknowledgement,Address,AssociatedDIFs,Association,Attribute Definition,Attribute Label,Author / Originator,...,Start Time,Temporal Extent,Theme Keyword,Topic Category,URL,Units,Unknown,VariableType,Web Page,Westernmost Longitude
0,Sample_Sample,dataset_3470.xml,1,2,5,2,2,17,17,1,...,0,1,15,1,1,17,477,1,7,1
1,Sample_Sample,dataset_3484.xml,1,2,5,2,2,5,5,1,...,0,1,6,1,1,5,352,1,7,1
2,Sample_Sample,dataset_3485.xml,1,2,5,2,2,14,14,1,...,0,1,14,1,1,14,429,1,7,1
3,Sample_Sample,dataset_3486.xml,1,2,5,2,2,16,16,1,...,0,1,15,1,1,16,455,1,7,1
4,Sample_Sample,dataset_3508.xml,1,2,5,2,2,10,10,1,...,0,1,10,1,1,10,398,1,7,1
5,Sample_Sample,dataset_3510.xml,1,2,5,2,2,11,11,1,...,0,1,11,1,1,11,408,1,7,1
6,Sample_Sample,dataset_3513.xml,1,2,5,2,2,11,11,1,...,0,1,10,1,1,11,395,1,7,1
7,Sample_Sample,dataset_3514.xml,1,2,5,2,2,10,10,1,...,0,1,9,1,1,10,385,1,7,1
8,Sample_Sample,dataset_3515.xml,1,2,5,2,2,11,11,1,...,0,1,10,1,1,11,395,1,7,1
9,Sample_Sample,dataset_3516.xml,1,2,5,2,2,27,27,1,...,0,1,24,1,1,27,584,1,7,1


In [42]:
FiledirectoryQuickE=os.path.join('../data/',Organization)
FilenameQuickE='/'+Collection+'_'+Dialect+'_QuickE.csv'
FilePathQuickE=FiledirectoryQuickE+FilenameQuickE
group_name = EvaluatedSimplifiedMetadataDF.groupby(['XPath', 'Record'], as_index=False)
QuickEdf=group_name.size().unstack().reset_index()
QuickEdf=QuickEdf.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
QuickEdf.to_csv(FilePathQuickE, mode = 'w', index=False)
QuickEdf

Record,XPath,dataset_3470.xml,dataset_3484.xml,dataset_3485.xml,dataset_3486.xml,dataset_3508.xml,dataset_3510.xml,dataset_3513.xml,dataset_3514.xml,dataset_3515.xml,...,dataset_647580.xml,dataset_647606.xml,dataset_647909.xml,dataset_648030.xml,dataset_648543.xml,dataset_648753.xml,dataset_650087.xml,dataset_650135.xml,dataset_650340.xml,dataset_651138.xml
0,/@xsi:schemaLocation,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,/acquisitionInformation/instrument,1,1,0,0,1,1,0,0,0,...,4,6,3,3,3,3,2,0,2,3
2,/acquisitionInformation/instrument/description,1,1,0,0,1,1,0,0,0,...,4,6,3,3,3,3,2,0,2,3
3,/acquisitionInformation/instrument/identifier/...,2,2,0,0,2,2,0,0,0,...,8,12,6,6,6,2,4,0,4,6
4,/acquisitionInformation/instrument/identifier/...,1,1,0,0,1,1,0,0,0,...,4,5,3,3,3,0,2,0,2,3
5,/acquisitionInformation/instrument/identifier/...,1,1,0,0,1,1,0,0,0,...,1,1,1,1,1,1,1,0,1,1
6,/acquisitionInformation/instrument/identifier/...,1,1,0,0,1,1,0,0,0,...,4,6,3,3,3,3,2,0,2,3
7,/acquisitionInformation/instrument/identifier/...,1,1,0,0,1,1,0,0,0,...,4,6,3,3,3,3,2,0,2,3
8,/acquisitionInformation/instrument/type,2,2,0,0,2,2,0,0,0,...,8,11,6,6,6,1,4,0,4,6
9,/acquisitionInformation/operation/description,2,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,3


In [43]:
FiledirectoryOccurance=os.path.join('../data/',Organization)
FilenameOccurance='/'+Collection+'_'+Dialect+'_Occurance.csv'
FilePathOccurance=FiledirectoryOccurance+FilenameOccurance

group_name = EvaluatedSimplifiedMetadataDF.groupby(['Record', 'Concept'], as_index=False)
occuranceMatrix=group_name.size().unstack().reset_index()
occuranceMatrix=occuranceMatrix.fillna(0)
occuranceSum=occuranceMatrix.sum()
occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
CollectionName=FilenameOccurance.partition("/")[2].partition("_Occurance.csv")[0]
result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
result.insert(1, 'Collection', CollectionName)
result.insert(4, 'CollectionOccurance%', CollectionName)
result.insert(4, 'AverageOccurancePerRecord', CollectionName)
result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
NumberOfRecords = result.at[0, 'ConceptCount'].count('.xml')
result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
result.at[0, 'ConceptCount'] = NumberOfRecords
result.at[0, 'Concept'] = 'Number of Records'
result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
result.to_csv(FilePathOccurance, mode = 'w', index=False)
result

,Concept,Collection,ConceptCount,RecordCount,AverageOccurancePerRecord,CollectionOccurance%
0,Number of Records,Sample_Sample,117,117,1.00,100.00%
1,Abstract,Sample_Sample,117,117,1.00,100.00%
2,Acknowledgement,Sample_Sample,191,115,1.63,98.29%
3,Address,Sample_Sample,592,117,5.06,100.00%
4,AssociatedDIFs,Sample_Sample,261,117,2.23,100.00%
5,Association,Sample_Sample,261,117,2.23,100.00%
6,Attribute Definition,Sample_Sample,4132,117,35.32,100.00%
7,Attribute Label,Sample_Sample,4385,117,37.48,100.00%
8,Author / Originator,Sample_Sample,125,117,1.07,100.00%
9,Bounding Box,Sample_Sample,88,88,0.75,75.21%


In [44]:
# Create dataframe of just the elements that do not have a version of Not Provided for their content
ContentProvidedDF = EvaluatedSimplifiedMetadataDF[EvaluatedSimplifiedMetadataDF.Content!=("Not provided" or "Not%20provided")]

if len(ContentProvidedDF)==len(EvaluatedSimplifiedMetadataDF):
   print("No elements contain a variant of 'Not provided' in their content for this collection")
   
else:
    print("Secondary data products, RAD, QuickE, Occurance, being created for collection for all elements that contain a variant of 'Not provided' in their content and a set of products for the elements that do not contain a variant of 'Not provided' in their content")
    
    # Create dataframe of just the elements that do not have a version of Not Provided for their content
    ContentProvidedDF = EvaluatedSimplifiedMetadataDF[EvaluatedSimplifiedMetadataDF.Content!=("Not provided" or "Not%20provided")]

    # Create secondary data products: RAD, QuickE, Occurance for both provided and not provided content.

    #not provided RAD
    FiledirectoryRADnotProvided=os.path.join('../data/',Organization)
    FilenameRADnotProvided='/'+Collection+'_'+Dialect+'_NotProvided_RAD.csv'
    FilePathRADnotProvided=FiledirectoryRADnotProvided+FilenameRADnotProvided
    group_namenotProvided = ContentNotProvidedDF.groupby(['Collection','Record', 'Concept'], as_index=False)
    occuranceMatrixnotProvided=group_namenotProvided.size().unstack().reset_index()
    occuranceMatrixnotProvided=occuranceMatrixnotProvided.fillna(0)
    pd.options.display.float_format = '{:,.0f}'.format
    occuranceMatrixnotProvided.to_csv(FilePathRADnotProvided, mode = 'w', index=False)

    #Provided RAD
    FiledirectoryRADProvided=os.path.join('../data/',Organization)
    FilenameRADProvided='/'+Collection+'_'+Dialect+'_Provided_RAD.csv'
    FilePathRADProvided=FiledirectoryRADProvided+FilenameRADProvided
    group_nameProvided = ContentProvidedDF.groupby(['Collection','Record', 'Concept'], as_index=False)
    occuranceMatrixProvided=group_nameProvided.size().unstack().reset_index()
    occuranceMatrixProvided=occuranceMatrixProvided.fillna(0)
    pd.options.display.float_format = '{:,.0f}'.format
    occuranceMatrixProvided.to_csv(FilePathRADProvided, mode = 'w', index=False)

    #not provided QuickE
    FiledirectoryQuickEnotProvided=os.path.join('../data/',Organization)
    FilenameQuickEnotProvided='/'+Collection+'_'+Dialect+'_NotProvided_QuickE.csv'
    FilePathQuickEnotProvided=FiledirectoryQuickEnotProvided+FilenameQuickEnotProvided
    group_namenotProvided = ContentNotProvidedDF.groupby(['XPath', 'Record'], as_index=False)
    QuickEdfnotProvided=group_namenotProvided.size().unstack().reset_index()
    QuickEdfnotProvided=QuickEdfnotProvided.fillna(0)
    pd.options.display.float_format = '{:,.0f}'.format
    QuickEdfnotProvided.to_csv(FilePathQuickEnotProvided, mode = 'w', index=False)

    #Provided QuickE
    FiledirectoryQuickEProvided=os.path.join('../data/',Organization)
    FilenameQuickEProvided='/'+Collection+'_'+Dialect+'_Provided_QuickE.csv'
    FilePathQuickEProvided=FiledirectoryQuickEProvided+FilenameQuickEProvided
    group_nameProvided = ContentProvidedDF.groupby(['XPath', 'Record'], as_index=False)
    QuickEdfProvided=group_nameProvided.size().unstack().reset_index()
    QuickEdfProvided=QuickEdfProvided.fillna(0)
    pd.options.display.float_format = '{:,.0f}'.format
    QuickEdfProvided.to_csv(FilePathQuickEProvided, mode = 'w', index=False)

    #Provided Occurance
    FiledirectoryOccurance=os.path.join('../data/',Organization)
    FilenameOccurance='/'+Collection+'_'+Dialect+'_Provided_Occurance.csv'
    FilePathOccurance=FiledirectoryOccurance+FilenameOccurance

    group_name = ContentProvidedDF.groupby(['Record', 'Concept'], as_index=False)
    occuranceMatrix=group_name.size().unstack().reset_index()
    occuranceMatrix=occuranceMatrix.fillna(0)
    occuranceSum=occuranceMatrix.sum()
    occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
    CollectionName=FilenameOccurance.partition("/")[2].partition("_Provided_Occurance.csv")[0]
    result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
    result.insert(1, 'Collection', CollectionName)
    result.insert(4, 'CollectionOccurance%', CollectionName)
    result.insert(4, 'AverageOccurancePerRecord', CollectionName)
    result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
    NumberOfRecords = result.at[0, 'ConceptCount'].count('.xml')
    result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
    result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
    result.at[0, 'ConceptCount'] = NumberOfRecords
    result.at[0, 'Concept'] = 'Number of Records'
    result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
    result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
    result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
    result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
    result.to_csv(FilePathOccurance, mode = 'w', index=False)
   
    #Not provided Occurance
    FiledirectoryOccurance=os.path.join('../data/',Organization)
    FilenameOccurance='/'+Collection+'_'+Dialect+'_NotProvided_Occurance.csv'
    FilePathOccurance=FiledirectoryOccurance+FilenameOccurance

    group_name = ContentNotProvidedDF.groupby(['Record', 'Concept'], as_index=False)
    occuranceMatrix=group_name.size().unstack().reset_index()
    occuranceMatrix=occuranceMatrix.fillna(0)
    occuranceSum=occuranceMatrix.sum()
    occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
    CollectionName=FilenameOccurance.partition("/")[2].partition("_NotProvided_Occurance.csv")[0]
    result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
    result.insert(1, 'Collection', CollectionName)
    result.insert(4, 'CollectionOccurance%', CollectionName)
    result.insert(4, 'AverageOccurancePerRecord', CollectionName)
    result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
    NumberOfRecords = result.at[0, 'ConceptCount'].count('.xml')
    result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
    result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
    result.at[0, 'ConceptCount'] = NumberOfRecords
    result.at[0, 'Concept'] = 'Number of Records'
    result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
    result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
    result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
    result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
    result.to_csv(FilePathOccurance, mode = 'w', index=False)
print("Good bye!")




No elements contain a variant of 'Not provided' in their content for this collection
Good bye!


### Combine different types of evaluated data to compare between collections or observe trends over time

* [Combine_Data](Combine_Data.ipynb)
